In [1]:
import config.display as dis
import config.radio as rad
import config.tv as tv
import sqlite3
import pandas as pd
import numpy as np
import matplotlib as mpl


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.copy_on_write = True

In [2]:
con_r = sqlite3.Connection(rad.DB_PATH)
cur_r = con_r.cursor()
con_tv = sqlite3.Connection(tv.DB_PATH)
cur_tv = con_tv.cursor()
con_d = sqlite3.Connection(dis.DB_PATH)
cur_d = con_d.cursor()

In [3]:
# Variables used to get proper data via SQL query
rok_s = 2022
rok_e = 2023
mc_s = 1
mc_e = 12
dzi_s = 1
dzi_e = 31

# Variables for YoY comparisons
rok_s_rdr = rok_s
rok_e_rdr = rok_e
mc_s_rdr = 'Luty'
mc_e_rdr = 'Luty'
dzi_s_rdr = 1
dzi_e_rdr = 26

# Variables for MoM comparisons
rok_s_mdm = rok_e_mdm = rok_e
mc_s_mdm = 'Luty'
mc_e_mdm = 'Styczeń'
dzi_s_mdm = 1
dzi_e_mdm = 26

# Variables for WoW comparisons
okres_obecny = ('2023-02-20', '2023-02-26')
okres_poprzedni = ('2023-02-13', '2023-02-19')

# Variables for column names creation for pivot tables
okr_poprzedni0 = '.'.join(okres_poprzedni[0].split('-')[::-1])
okr_poprzedni1 = '.'.join(okres_poprzedni[1].split('-')[::-1])
okr_obecny0 = '.'.join(okres_obecny[0].split('-')[::-1])
okr_obecny1 = '.'.join(okres_obecny[1].split('-')[::-1])

In [4]:
query_r = f'''SELECT "data_czas"."data" AS "data", "dzien", "dzien_tyg_nr" AS 'd_tyg_nr', "dzien_tyg", 
"tydzien", "miesiac_nr" AS "m_nr", "miesiac", "rok", "kod_rek" AS "kod_reklamy", "brand", 
"submedium", "nadawca", "zasieg", "godz_blok_rek" AS "godz_bloku", "daypart", 
"dl_ujednolicona" AS "dl_spotu", "typ_produktu", "koszt", "typ_rek", "l_emisji" AS "ilosc"
FROM "spoty"
JOIN "data_czas" ON "data_czas"."data" = "spoty"."data"
JOIN "brandy" ON "brandy"."id" = "spoty"."brand_id"
JOIN "submedia" ON "submedia"."id" = "spoty"."submedium_id"
JOIN "nadawcy" ON "nadawcy"."id" = "submedia"."nadawca_id"
JOIN "zasiegi" ON "zasiegi"."id" = "submedia"."zasieg_id"
JOIN "dayparty" ON "dayparty"."id" = "spoty"."daypart_id"
JOIN "dl_ujednolicone" ON "dl_ujednolicone"."id" = "spoty"."dl_ujednolicona_id"
JOIN "typy_produktu" ON "typy_produktu"."id" = "spoty"."typ_produktu_id"
JOIN "dni_tyg" ON "dni_tyg"."id" = "data_czas"."dzien_tyg_nr"
JOIN "miesiace" ON "miesiace"."id" = "data_czas"."miesiac_nr"
JOIN "bloki_rek" ON "bloki_rek"."id" = "spoty"."blok_rek_id"
JOIN "kody_rek" ON "kody_rek"."id" = "spoty"."kod_rek_id"
JOIN "typy_rek" ON "typy_rek"."id" = "spoty"."typ_rek_id"
WHERE "miesiac_nr" BETWEEN {mc_s} AND {mc_e} AND "rok" BETWEEN {rok_s} AND {rok_e} 
AND "dzien" BETWEEN {dzi_s} AND {dzi_e};'''


query_tv = f'''SELECT data_czas.data AS data, dzien, dzien_tyg_nr AS d_tyg_nr, dzien_tyg,
tydzien, miesiac_nr AS m_nr, miesiac, rok, kod_rek AS kod_reklamy, brand, kanal, 
kanal_gr, czas, dlugosc, daypart, grp AS ilosc
FROM spoty
JOIN miesiace ON miesiace.id = data_czas.miesiac_nr
JOIN dni_tyg ON dni_tyg.id = data_czas.dzien_tyg_nr
JOIN data_czas ON data_czas.data = spoty.data
JOIN kody_rek ON kody_rek.id = spoty.kod_rek_id
JOIN dlugosci ON dlugosci.id = spoty.dlugosc_id
JOIN dayparty ON dayparty.id = spoty.daypart_id
JOIN brandy ON brandy.id = spoty.brand_id
JOIN kanaly ON kanaly.id = spoty.kanal_id
JOIN kanaly_gr ON kanaly_gr.id = kanaly.kanal_gr_id
WHERE m_nr BETWEEN {mc_s} AND {mc_e} AND rok BETWEEN {rok_s} AND {rok_e} 
AND dzien BETWEEN {dzi_s} AND {dzi_e};'''

query_d = f'''SELECT data_czas.data AS data, dzien, dzien_tyg_nr AS d_tyg_nr, dzien_tyg,
tydzien, miesiac_nr AS m_nr, miesiac, rok, wydawca, format, urzadzenie, brand, pv AS ilosc, 
CAST(1 AS INTEGER) AS liczba
FROM placementy
JOIN miesiace ON miesiace.id = data_czas.miesiac_nr
JOIN dni_tyg ON dni_tyg.id = data_czas.dzien_tyg_nr
JOIN data_czas ON data_czas.data = placementy.data
JOIN wydawcy ON placementy.wydawca_id = wydawcy.id
JOIN formaty ON placementy.format_id = formaty.id
JOIN urzadzenia ON placementy.urzadzenie_id = urzadzenia.id
JOIN brandy ON placementy.brand_id = brandy.id
WHERE m_nr BETWEEN {mc_s} AND {mc_e} AND rok BETWEEN {rok_s} AND {rok_e} 
AND dzien BETWEEN {dzi_s} AND {dzi_e};'''

In [5]:
def set_period(x: str) -> str:
    """
    Function used for setting period name for WoW comparison. Applies for every row that fits.
    """
    if okres_poprzedni[1] >= x >= okres_poprzedni[0]:
        if okres_poprzedni[0][:7] == okres_poprzedni[1][:7]:
            return f'1. {okr_poprzedni0[:2]}-{okr_poprzedni1}'
        elif okres_poprzedni[0][:7] != okres_poprzedni[1][:7] and okres_poprzedni[0][:4] == okres_poprzedni[1][:4]:
            return f'1. {okr_poprzedni0[:5]}-{okr_poprzedni1}'
        else:
            return f'1. {okr_poprzedni0}-{okr_poprzedni1}'
    if okres_obecny[1] >= x >= okres_obecny[0]:
        if okres_obecny[0][:7] == okres_obecny[1][:7]:
            return f'2. {okr_obecny0[:2]}-{okr_obecny1}'
        elif okres_obecny[0][:7] != okres_obecny[1][:7] and okres_obecny[0][:4] == okres_obecny[1][:4]:
            return f'2. {okr_obecny0[:5]}-{okr_obecny1}'
        else:
            return f'2. {okr_obecny0}-{okr_obecny1}'


def extract_period(start: str, end: str) -> str:
    """
    Extracts period for WoW column names analysis
    """
    if start[3:] == end[3:]:
        return f'{start[:2]}-{end}'
    elif start[3:] != end[1][3:] and start[-4:] == end[-4:]:
        return f'{start[:5]}-{end}'
    else:
        return f'{start}-{end}'


def swap_month(x: str) -> str:
    """
    Swaps names of months for columns in MoM analysis
    """
    if x == mc_s_mdm:
        return f'1. {mc_s_mdm}'
    elif x == mc_e_mdm:
        return f'2. {mc_e_mdm}'


In [6]:
df_r = pd.read_sql(sql=query_r, con=con_r)
df_r.head(1)

,data,dzien,d_tyg_nr,dzien_tyg,tydzien,m_nr,miesiac,rok,kod_reklamy,brand,submedium,nadawca,zasieg,godz_bloku,daypart,dl_spotu,typ_produktu,koszt,typ_rek,ilosc
0,2022-01-01,1,6,Sobota,0,1,Styczeń,2022,22126226,EURO RTV AGD,PR 1,POLSKIE RADIO,krajowe,6:00-6:29,do 9,30,GRUPA,1700,reklama,1


In [7]:
df_tv = pd.read_sql(sql=query_tv, con=con_tv)
df_tv.head(1)

,data,dzien,d_tyg_nr,dzien_tyg,tydzien,m_nr,miesiac,rok,kod_reklamy,brand,kanal,kanal_gr,czas,dlugosc,daypart,ilosc
0,2022-01-01,1,6,Sobota,0,1,Styczeń,2022,290969,MEDIA EXPERT,TV6,Polsat tem,18:43:39,15,prime,0.49


In [8]:
df_d = pd.read_sql(sql=query_d, con=con_d)
df_d.head(1)

,data,dzien,d_tyg_nr,dzien_tyg,tydzien,m_nr,miesiac,rok,wydawca,format,urzadzenie,brand,ilosc,liczba
0,2022-01-01,1,6,Sobota,0,1,Styczeń,2022,WP,RECTANGLE 3XUU| M,M,MEDIA EXPERT,5400000,1


In [9]:
dziennie_r = df_r[['brand', 'submedium', 'ilosc', 'dzien', 'dzien_tyg', 'data', 'rok', 'miesiac']]
dziennie_r.loc[:, 'dzien_tyg'] = dziennie_r['dzien_tyg'].str[:2]
dziennie_r['medium'] = 'radio'

dziennie_t = df_tv[['brand', 'ilosc', 'dzien', 'dzien_tyg', 'data', 'rok', 'miesiac']]
dziennie_t.loc[:, 'dzien_tyg'] = dziennie_t['dzien_tyg'].str[:2]
dziennie_t['medium'] = 'tv'

dziennie_d = df_d[['brand', 'ilosc', 'liczba', 'dzien', 'dzien_tyg', 'data', 'rok', 'miesiac']]
dziennie_d.loc[:, 'dzien_tyg'] = dziennie_d['dzien_tyg'].str[:2]
dziennie_d['medium'] = 'display'


# YoY
stacje = ['ESKA WROCŁAW', 'ZET', 'RMF FM'] #['ESKA Wrocław', 'ZET', 'RMF FM', 'TOK FM', 'PR 1']
rok_rdr = (rok_s_rdr, rok_e_rdr)
miesiac_rdr = (mc_s_rdr, mc_e_rdr)
dzien_rdr = dzi_e_rdr

filtr_r_rdr = (dziennie_r['submedium'].isin(stacje)) & (dziennie_r['rok'].isin(rok_rdr)) & (dziennie_r['miesiac'].isin(miesiac_rdr)) \
              & (dziennie_r['dzien'] <= dzien_rdr)
dziennie_r_rdr = dziennie_r.loc[filtr_r_rdr]
dziennie_r_rdr.set_index(['brand', 'medium'], inplace=True)
dziennie_r_rdr = dziennie_r_rdr.sort_index(ascending=True)

filtr_t_rdr = (dziennie_t['rok'].isin(rok_rdr)) & (dziennie_t['miesiac'].isin(miesiac_rdr)) & (dziennie_t['dzien'] <= dzien_rdr)
dziennie_t_rdr = dziennie_t[filtr_t_rdr]
dziennie_t_rdr.set_index(['brand', 'medium'], inplace=True)
dziennie_t_rdr = dziennie_t_rdr.sort_index(ascending=True)

filtr_d_rdr = (dziennie_d['rok'].isin(rok_rdr)) & (dziennie_d['miesiac'].isin(miesiac_rdr)) & (dziennie_d['dzien'] <= dzien_rdr)
dziennie_d_rdr = dziennie_d[filtr_d_rdr]
dziennie_d_rdr.set_index(['brand', 'medium'], inplace=True)
dziennie_d_rdr = dziennie_d_rdr.sort_index(ascending=True)

tp_r_rdr = dziennie_r_rdr.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_t_rdr = dziennie_t_rdr.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_d_pv_rdr = dziennie_d_rdr.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_d_pv_rdr.index = tp_d_pv_rdr.index.set_levels(tp_d_pv_rdr.index.levels[1].str.replace('display', 'display pv'), level=1)
tp_d_rdr = dziennie_d_rdr.pivot_table(values='liczba', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_d_rdr.index = tp_d_rdr.index.set_levels(tp_d_rdr.index.levels[1].str.replace('display', 'display ilość'), level=1)


# MoM
rok_mdm = (rok_s_mdm, rok_e_mdm)
miesiac_mdm = (mc_s_mdm, mc_e_mdm)
dzien_mdm = (dzi_s_mdm, dzi_e_mdm)

filtr_r_mdm = (dziennie_r['submedium'].isin(stacje)) & (dziennie_r['rok'].isin(rok_mdm)) & (dziennie_r['miesiac'].isin(miesiac_mdm)) \
          & (dziennie_r['dzien'] <= dzien_mdm[1]) & (dziennie_r['dzien'] >= dzien_mdm[0])
dziennie_r_mdm = dziennie_r.loc[filtr_r_mdm]
dziennie_r_mdm['miesiac'] = dziennie_r_mdm['miesiac'].apply(swap_month)
dziennie_r_mdm.set_index(['brand', 'medium'], inplace=True)
dziennie_r_mdm = dziennie_r_mdm.sort_index(ascending=True)

filtr_t_mdm = (dziennie_t['rok'].isin(rok_mdm)) & (dziennie_t['miesiac'].isin(miesiac_mdm)) & (dziennie_t['dzien'] <= dzien_mdm[1]) \
          & (dziennie_t['dzien'] >= dzien_mdm[0])
dziennie_t_mdm = dziennie_t[filtr_t_mdm]
dziennie_t_mdm['miesiac'] = dziennie_t_mdm['miesiac'].apply(swap_month)
dziennie_t_mdm.set_index(['brand', 'medium'], inplace=True)
dziennie_t_mdm = dziennie_t_mdm.sort_index(ascending=True)

filtr_d_mdm = (dziennie_d['rok'].isin(rok_mdm)) & (dziennie_d['miesiac'].isin(miesiac_mdm)) & (dziennie_d['dzien'] <= dzien_mdm[1]) \
          & (dziennie_d['dzien'] >= dzien_mdm[0])
dziennie_d_mdm = dziennie_d[filtr_d_mdm]
dziennie_d_mdm['miesiac'] = dziennie_d_mdm['miesiac'].apply(swap_month)
dziennie_d_mdm.set_index(['brand', 'medium'], inplace=True)
dziennie_d_mdm = dziennie_d_mdm.sort_index(ascending=True)


tp_r_mdm = dziennie_r_mdm.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_t_mdm = dziennie_t_mdm.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_d_pv_mdm = dziennie_d_mdm.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_d_pv_mdm.index = tp_d_pv_mdm.index.set_levels(tp_d_pv_mdm.index.levels[1].str.replace('display', 'display pv'), level=1)
tp_d_mdm = dziennie_d_mdm.pivot_table(values='liczba', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc='sum')
tp_d_mdm.index = tp_d_mdm.index.set_levels(tp_d_mdm.index.levels[1].str.replace('display', 'display ilość'), level=1)


# WoW
filtr_r_tdt = ((dziennie_r['submedium'].isin(stacje)) & (okres_obecny[1] >= dziennie_r['data']) & (dziennie_r['data'] >= okres_obecny[0])) | \
                ((dziennie_r['submedium'].isin(stacje)) & (okres_poprzedni[1] >= dziennie_r['data']) & (dziennie_r['data'] >= okres_poprzedni[0]))
dziennie_r_tdt = dziennie_r.loc[filtr_r_tdt]
dziennie_r_tdt['okres'] = dziennie_r_tdt['data'].apply(set_period)
dziennie_r_tdt.set_index(['brand', 'medium'], inplace=True)
dziennie_r_tdt = dziennie_r_tdt.sort_index(ascending=True)

filtr_t_tdt = ((dziennie_t['data'] >= okres_obecny[0]) & (dziennie_t['data'] <= okres_obecny[1])) | \
                ((dziennie_t['data'] >= okres_poprzedni[0]) & (dziennie_t['data'] <= okres_poprzedni[1]))
dziennie_t_tdt = dziennie_t[filtr_t_tdt]
dziennie_t_tdt['okres'] = dziennie_t_tdt['data'].apply(set_period)
dziennie_t_tdt.set_index(['brand', 'medium'], inplace=True)
dziennie_t_tdt = dziennie_t_tdt.sort_index(ascending=True)

filtr_d_tdt = ((dziennie_d['data'] >= okres_obecny[0]) & (dziennie_d['data'] <= okres_obecny[1])) | \
                ((dziennie_d['data'] >= okres_poprzedni[0]) & (dziennie_d['data'] <= okres_poprzedni[1]))
dziennie_d_tdt = dziennie_d[filtr_d_tdt]
dziennie_d_tdt['okres'] = dziennie_d_tdt['data'].apply(set_period)
dziennie_d_tdt.set_index(['brand', 'medium'], inplace=True)
dziennie_d_tdt = dziennie_d_tdt.sort_index(ascending=True)

tp_r_tdt = dziennie_r_tdt.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['okres'], aggfunc='sum')
tp_t_tdt = dziennie_t_tdt.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['okres'], aggfunc='sum')
tp_d_pv_tdt = dziennie_d_tdt.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['okres'], aggfunc='sum')
tp_d_pv_tdt.index = tp_d_pv_tdt.index.set_levels(tp_d_pv_tdt.index.levels[1].str.replace('display', 'display pv'), level=1)
tp_d_tdt = dziennie_d_tdt.pivot_table(values='liczba', index=['brand', 'medium'], columns=['okres'], aggfunc='sum')
tp_d_tdt.index = tp_d_tdt.index.set_levels(tp_d_tdt.index.levels[1].str.replace('display', 'display ilość'), level=1)


In [10]:
# tp_rdr = tp_r_rdr.merge(tp_t_rdr, how='outer', on=['brand', 'medium'])
# tp_rdr2 = tp_d_pv_rdr.merge(tp_d_rdr, how='outer', on=['brand', 'medium'])
# tp_rdr = tp_rdr.merge(tp_rdr2, how='outer', on=['brand', 'medium'])
tp_rdr = pd.concat([tp_r_rdr, tp_t_rdr, tp_d_rdr, tp_d_pv_rdr], axis=0)
tp_rdr = tp_rdr.groupby(['brand', 'medium']).sum()
tp_rdr['roznica'] = tp_rdr[rok_rdr[1]] - tp_rdr[rok_rdr[0]]
tp_rdr['dynamika'] = tp_rdr[rok_rdr[1]] / tp_rdr[rok_rdr[0]] - 1
s = tp_rdr.style \
    .format('{:,.0f}', thousands=' ', decimal=',') \
    .format('{:,.1%}', thousands=' ', decimal=",", subset='dynamika')
s




In [11]:
# tp_mdm = tp_r_mdm.merge(tp_t_mdm, how='outer', on=['brand', 'medium'])
tp_mdm = pd.concat([tp_r_mdm, tp_t_mdm, tp_d_mdm, tp_d_pv_mdm], axis=0)
tp_mdm = tp_mdm.groupby(['brand', 'medium']).sum()
tp_mdm['roznica'] = tp_mdm[rok_mdm[1]][f'2. {miesiac_mdm[1]}'] - tp_mdm[rok_mdm[0]][f'1. {miesiac_mdm[0]}']
tp_mdm['dynamika'] = tp_mdm[rok_mdm[1]][f'2. {miesiac_mdm[1]}'] / tp_mdm[rok_mdm[0]][f'1. {miesiac_mdm[0]}'] - 1
s = tp_mdm.style \
    .format('{:,.0f}', thousands=' ', decimal=',') \
    .format('{:,.1%}', thousands=' ', decimal=",", subset='dynamika')
s

In [12]:
# tp_tdt = tp_r_tdt.merge(tp_t_tdt, how='outer', on=['brand', 'medium'])
start = f'1. {extract_period(okr_poprzedni0, okr_poprzedni1)}'
end =  f'2. {extract_period(okr_obecny0, okr_obecny1)}'

tp_tdt = pd.concat([tp_r_tdt, tp_t_tdt, tp_d_tdt, tp_d_pv_tdt], axis=0)
tp_tdt = tp_tdt.groupby(['brand', 'medium']).sum()
tp_tdt['roznica'] = tp_tdt[end] - tp_tdt[start]
tp_tdt['dynamika'] = tp_tdt[end] / tp_tdt[start] - 1
# tp_tdt['roznica'] = tp_tdt[rok_tdt[1]][miesiac_tdt[1]] - tp_tdt[rok_tdt[0]][miesiac_tdt[0]]
# tp_tdt['dynamika'] = tp_tdt[rok_tdt[1]][miesiac_tdt[1]] / tp_mdm[rok_tdt[0]][miesiac_tdt[0]] - 1
s = tp_tdt.style \
    .format('{:,.0f}', thousands=' ', decimal=',') \
    .format('{:,.1%}', thousands=' ', decimal=",", subset='dynamika')
s

In [ ]:
# To file extraction
with pd.ExcelWriter('../../Aktywność Konkurencji - porównania/srednie_pv.xlsx', mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
    tp_rdr.to_excel(excel_writer=writer, sheet_name='rdr')
    tp_mdm.to_excel(excel_writer=writer, sheet_name='mdm')
    tp_tdt.to_excel(excel_writer=writer, sheet_name='tdt')